# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("output_data/cities.csv")
weather_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Puerto Ayora,-0.74,-90.35,68.00,96,79,5.99,EC,1595556616
1,1,Szentgotthárd,46.95,16.27,63.00,71,74,1.99,HU,1595556747
2,2,Malaṅgawā,26.86,85.56,83.41,75,78,10.60,NP,1595556614
3,3,Mizdah,31.45,12.98,62.35,81,0,3.09,LY,1595556747
4,4,Payson,40.04,-111.73,77.00,33,75,13.87,US,1595556747
...,...,...,...,...,...,...,...,...,...,...
586,586,Voi,-3.39,38.56,62.51,84,14,5.88,KE,1595556814
587,587,Bambanglipuro,-7.95,110.28,80.56,67,98,12.97,ID,1595556814
588,588,Saint-Raphaël,19.43,-72.20,73.92,84,84,4.09,HT,1595556814
589,589,Jumla,29.27,82.18,66.00,62,68,0.65,NP,1595556814


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:

fig = gmaps.figure(map_type='SATELLITE')
heatmap_locs = weather_data[["Lat","Lng"]]
humidity= weather_data["Humidity"].astype(float)
maxintensity = humidity.max()

heatmap_layer= gmaps.heatmap_layer(
    heatmap_locs,weights=humidity,max_intensity=maxintensity, point_radius =2,dissipating=False)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature < 80 degrees but > 70.
# Wind speed < 10mph
# Cloudiness = 0
# drop any rows that don't contain any of the above


ideal_temp = weather_data.loc[
    (weather_data["Max Temp"]<80) & 
    (weather_data["Max Temp"]>70) & 
    (weather_data["Wind Speed"]<10) & 
    (weather_data["Cloudiness"]==0)]


# Dropping any null values
hotel_df = ideal_temp.dropna(how='any')
hotel_df.reset_index(inplace=True)
hotel_df




,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,49,49,Jurm,36.86,70.83,75.42,24,0,1.32,AF,1595556664
1,54,54,Manbij,36.53,37.95,75.20,57,0,9.31,SY,1595556752
2,75,75,Lac du Bonnet,50.25,-96.06,75.00,99,0,8.01,CA,1595556754
3,157,157,Šibenik,43.73,15.91,73.40,69,0,4.36,HR,1595556764
4,158,158,Zarand,30.81,56.56,70.34,22,0,4.21,IR,1595556764
5,165,165,Sirte,31.21,16.59,75.40,82,0,6.60,LY,1595556765
6,201,201,Kikwit,-5.04,18.82,73.98,37,0,0.45,CD,1595556769
7,302,302,Cariati,39.49,16.96,71.01,66,0,5.77,IT,1595556781
8,316,316,Kiryat Gat,31.61,34.76,75.00,62,0,5.03,IL,1595556725
9,319,319,Sabzawār,36.21,57.68,77.49,47,0,7.58,IR,1595556783


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotels = []

distance=5000

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Grabbing the location based on Lat and Lng in the ideal_hotels df
for location in range(len(hotel_df)):
    latitude = hotel_df.loc[location]["Lat"]
    longitude = hotel_df.loc[location]["Lng"]

    params = {"location":f"{latitude},{longitude}",
          "radius": distance,
          "type":"hotel",
          "key":g_key}
    request = requests.get(url,params)
    request_json = request.json()
   
    try:
        hotels.append(request_json["results"][0]["name"])
    except:
        hotels.append("No hotel near")


        
hotel_df
# Adding Hotel Name column to "hotel_df"


hotel_df["Hotel Name"] = hotels

hotel_df

# print(request)    
# print(request.url)


,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,49,49,Jurm,36.86,70.83,75.42,24,0,1.32,AF,1595556664,Jurm
1,54,54,Manbij,36.53,37.95,75.20,57,0,9.31,SY,1595556752,Manbij
2,75,75,Lac du Bonnet,50.25,-96.06,75.00,99,0,8.01,CA,1595556754,Lac du Bonnet
3,157,157,Šibenik,43.73,15.91,73.40,69,0,4.36,HR,1595556764,Šibenik
4,158,158,Zarand,30.81,56.56,70.34,22,0,4.21,IR,1595556764,Zarand
5,165,165,Sirte,31.21,16.59,75.40,82,0,6.60,LY,1595556765,Sirte
6,201,201,Kikwit,-5.04,18.82,73.98,37,0,0.45,CD,1595556769,Kikwit
7,302,302,Cariati,39.49,16.96,71.01,66,0,5.77,IT,1595556781,Cariati
8,316,316,Kiryat Gat,31.61,34.76,75.00,62,0,5.03,IL,1595556725,Kiryat Gat
9,319,319,Sabzawār,36.21,57.68,77.49,47,0,7.58,IR,1595556783,Sabzevar


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map

hotel_markers = gmaps.marker_layer(locations)
fig.add_layer(hotel_markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))